In [19]:
import pickle as pkl
import numpy as np
import os
import torch



os.chdir('/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/loaders')
from create_dataloader_sequential import (load_course_encoder)

os.chdir('/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/architectures/Sequence')
from sec_transformer_pytorch import (create_model, load_pytorch_weights)

os.chdir('/home/jcsanguino10/local_citation_model/recommender-fusion-recsys/architectures/Multimodal')
from multimodal import Autoencoder, MultimodalModel

In [20]:
if torch.cuda.is_available():
    # Change to just a particular GPU changing the enviroment variable
    os.environ["CUDA_VISIBLE_DEVICES"] = "3"
    device = torch.device("cuda")
    # Change to just use a particular GPU via torch
    #torch.cuda.set_device("cuda:3")
    print(torch.cuda.current_device())

0


In [21]:
# Paths from the label encoder dicts 

PATH_TO_LABEL_ENCODER = '/home/jcsanguino10/local_citation_model/data/processed/'

# Path to folder with datasets

PATH_TO_DATASETS = '/home/jcsanguino10/local_citation_model/data/'

# Path to folder with checkpoints best models

PATH_TO_CHECKPOINTS = '/home/jcsanguino10/local_citation_model/models/'

In [22]:
label_encoder, dicts = load_course_encoder('/home', PATH_TO_LABEL_ENCODER)

Loading existing mappings from /home/jcsanguino10/local_citation_model/data/processed


In [23]:
len(label_encoder.classes_)  # List of all course IDs in the dataset

197

In [ ]:
import pandas as pd
#Change for train sufix or test sufix 
df_binary = pd.read_pickle(f'{PATH_TO_DATASETS}test_binary_all_vectors_128_01_transe.pkl')
df_bpr_df = pd.read_pickle(f'{PATH_TO_DATASETS}test_bpr_all_vectors_128_01_transe.pkl')

In [25]:
df_bpr_df.head()

,user_id,pos_item_id,neg_item_id,item_seq,pos_item_text_embedding,neg_item_text_embedding,user_text_embedding,user_cum_text_embedding,pos_item_bpr_embedding,neg_item_bpr_embedding,user_bpr_embedding,pos_item_graph_embedding,neg_item_graph_embedding,user_graph_embedding
0,0,6897,7001,[6897],"[0.00029639987, 0.014771659, 0.014058901, -0.0...","[-0.021708943, -0.07313048, 0.0040764194, 0.01...","[-0.022005824, -0.033391304, -0.013403211, -0....","[0.00029639987, 0.014771659, 0.014058901, -0.0...","[-0.004581114, 0.0626458, -0.02941907, -0.0164...","[0.02627153, -0.042739343, 0.0686576, 0.071189...","[-0.16485311, 0.13305487, -0.109800704, -0.159...","[-0.019909197, 0.015652379, -0.18591776, -0.19...","[-0.09981072, -0.17273627, -0.06508206, -0.185...","[-0.21868221, 0.107340455, 0.20054282, 0.15184..."
1,1,6920,6998,[6920],"[-0.001356997, -0.027733184, 0.010143433, -0.0...","[0.06441419, 0.036574744, 0.0004071652, -0.046...","[0.034128785, -0.054559205, 0.0015806267, -0.0...","[-0.001356997, -0.027733184, 0.010143433, -0.0...","[-0.031264078, 0.065558866, 0.002417838, -0.05...","[0.11860577, -0.10629058, 0.09773831, 0.059424...","[-0.15681404, 0.14787957, -0.14133789, -0.1282...","[0.21109915, 0.16339235, 0.031736396, -0.01513...","[-0.033010166, -0.1609984, -0.0942061, -0.1157...","[0.16515407, 0.18278724, -0.07278056, -0.06131..."
2,2,6937,6992,[6937],"[0.009646171, 0.015768941, 0.0102583, 0.011016...","[0.030553505, -0.08276028, 0.021069285, -0.022...","[0.017988527, -0.038172964, -0.008762296, -0.0...","[0.009646171, 0.015768941, 0.0102583, 0.011016...","[-0.048642207, 0.019007254, -0.033599593, -0.0...","[0.10899715, -0.09564237, 0.10501876, 0.085825...","[-0.12063337, 0.13376932, -0.13144864, -0.0975...","[-0.07782954, 0.19538878, 0.08653913, -0.04023...","[0.041632142, 0.070655555, -0.13588788, -0.160...","[-0.21761492, 0.1274874, 0.19495715, -0.179376..."
3,2,6961,7043,"[6937, 6961]","[0.03704412, -0.011753682, 0.0050762245, 0.021...","[-0.007348519, -0.056608632, -0.011322977, 0.0...","[0.017988527, -0.038172964, -0.008762296, -0.0...","[0.023345144, 0.0020076297, 0.007667262, 0.016...","[-0.014944556, -0.028715644, 0.0095228795, 0.0...","[0.120994814, -0.089827366, 0.10192869, 0.0929...","[-0.12063337, 0.13376932, -0.13144864, -0.0975...","[0.16374268, -0.07315293, 0.10547099, 0.158405...","[0.03657927, -0.060817167, -0.195846, -0.21245...","[-0.21761492, 0.1274874, 0.19495715, -0.179376..."
4,2,6895,7040,"[6937, 6961, 6895]","[-0.031121682, -0.08368267, 8.339915e-05, 0.06...","[-0.004670808, -0.05079265, 0.0073155914, -0.0...","[0.017988527, -0.038172964, -0.008762296, -0.0...","[0.0051895357, -0.026555805, 0.0051393076, 0.0...","[-0.073091574, 0.10878877, -0.07272676, -0.079...","[0.09851998, -0.10024271, 0.09955378, 0.079989...","[-0.12063337, 0.13376932, -0.13144864, -0.0975...","[0.18731615, 0.13930675, 0.13596186, 0.1819212...","[-0.022520965, 0.16019231, -0.16499598, -0.088...","[-0.21761492, 0.1274874, 0.19495715, -0.179376..."


In [26]:
# Create a new column that contains per each user the most longer sequence of courses taken for the user and add in each row the same sequence
def add_longest_sequence_column(df, user_col, seq_col, new_col):
    longest_seqs = df.groupby(user_col)[seq_col].apply(lambda x: max(x, key=len)).to_dict()
    df[new_col] = df[user_col].map(longest_seqs)
    return df

df_bpr_df = add_longest_sequence_column(df_bpr_df, 'user_id', 'item_seq', 'full_item_seq')
df_binary = add_longest_sequence_column(df_binary, 'user_id', 'item_seq', 'full_item_seq')

In [27]:
dicts.keys()

dict_keys(['filtered_id2entity', 'entity2newid', 'newid2entity', 'newid2oldid'])

In [28]:
## Transform values to keys and viceversa to create a mapping from old course IDs to new course IDs using dicts["newid2oldid"]
newid2oldid = dicts["newid2oldid"]
oldid2newid = {v: k for k, v in newid2oldid.items()}    

In [29]:
def parse_column_with_mapping(df, column_name, mapping, new_column_name=None):
    """
    Parses the elements in a specified column of a DataFrame using a mapping dictionary
    and optionally saves the results in a new column.

    Args:
        df (pd.DataFrame): The DataFrame containing the column to parse.
        column_name (str): The name of the column to parse.
        mapping (dict): A dictionary mapping old IDs to new IDs.
        new_column_name (str, optional): The name of the new column to save the results. 
                                         If None, the original column is overwritten.

    Returns:
        pd.DataFrame: A DataFrame with the specified column parsed using the mapping.
    """
    parsed_column = df[column_name].apply(lambda x: [mapping.get(item, item) for item in x])
    if new_column_name:
        df[new_column_name] = parsed_column
    else:
        df[column_name] = parsed_column
    return df

In [30]:
df_bpr_df = parse_column_with_mapping(df_bpr_df, 'full_item_seq', oldid2newid, new_column_name='parsed_item_seq')
df_binary = parse_column_with_mapping(df_binary, 'full_item_seq', oldid2newid, new_column_name='parsed_item_seq')


In [31]:
model = create_model(len(label_encoder.classes_) + 1, embedding_dim=128, 
                         num_heads=2, ff_dim=512, num_transformer_blocks=3, dropout_rate=0.1, padding_idx=len(label_encoder.classes_))

In [32]:
model = load_pytorch_weights(model, f'{PATH_TO_CHECKPOINTS}SEQUENTIAL_AUGMENT_2h_3b_best.pth')

PyTorch model weights loaded from /home/jcsanguino10/local_citation_model/models/SEQUENTIAL_AUGMENT_2h_3b_best.pth


In [33]:
## Create a function that take the sequence of course IDs from a column in a dataframe and generate a new column with the sequence vector using the trained model
def add_sequence_vectors(df, sequence_column, model, device):
    ## Add tqdm for progress bar
    from tqdm import tqdm
    tqdm.pandas()
    df['user_sequence_embedding'] = df[sequence_column].progress_apply(lambda x: generate_sequence_vector(model, x, device))
    return df
def generate_sequence_vector(model, course_sequence, device):
    model.eval()
    with torch.no_grad():
        # Convert course_sequence to tensor and add batch dimension
        input_tensor = torch.tensor(course_sequence, dtype=torch.long).unsqueeze(0).to(device)
        # Get the model output
        output = model(input_tensor)
        # Convert output to numpy array and remove batch dimension
        sequence_vector = output.squeeze(0).cpu().numpy()
    return sequence_vector

In [34]:
df_bpr_df = add_sequence_vectors(df_bpr_df, 'parsed_item_seq', model, 'cuda')

100%|██████████| 11924/11924 [00:27<00:00, 433.17it/s]


In [35]:
df_binary = add_sequence_vectors(df_binary, 'parsed_item_seq', model, 'cuda')

100%|██████████| 23848/23848 [00:54<00:00, 437.03it/s]


In [36]:
## save the dataframes with the new column with the sequence vectors
#Change for train sufix or test sufix 
df_bpr_df.to_pickle(f'{PATH_TO_DATASETS}test_bpr_all_vectors_128_01_transe_seqvec.pkl')
df_binary.to_pickle(f'{PATH_TO_DATASETS}test_binary_all_vectors_128_01_transe_seqvec.pkl')